In [1]:
# from src.utils.functions import (
#     doc_conv, check_ocr, free_memory, format_msgs, pdf_loader,
#     modify_orig, delete_papers, get_responses , post_request_with_retries, prompt_prep,
#     resize_image, data_preparation, models_local, models_used,get_img_summary, qdrant_process,
#     api_models_one_img, save_to_pickle, upsert_to_qdrant,colpali_qdrant,
#     encode_image, show_results, convert_pdfs_to_images,
#     check_vllm_status, monitor_vllm_process, retrieve_colpali,process_models,
#     update_vd_new_user,make_tarfile, extract_tarfile, new_user_set_files
#     #CustomRetriever,summarise_context,api_models_imgs

# )
from dotenv import load_dotenv, find_dotenv
load_dotenv()

from functions import (
    pdf_loader, process_models
)


import docling
from transformers import AutoTokenizer
from langchain_huggingface import HuggingFaceEmbeddings
import os

import torch
torch.classes.__path__ = []
import onnxruntime as rt

from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore


In [2]:
OPEN_MODEL = "google/gemma-3-27b-it"
OPEN_MODEL_SHORT = "Gemma3-27b"

LI_MODEL = "vidore/colpali-v1.3-merged"
LI_MODEL_SHORT = "ColPali"

EMBED_MODEL_ID = "BAAI/bge-base-en-v1.5"
MAX_TOKENS = 512
EMB_DIM = 768
VECTOR_SIZE=128

embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL_ID,
    model_kwargs={"device": "cuda"}
)
emb_tokenizer = AutoTokenizer.from_pretrained(EMBED_MODEL_ID )

PAPERS_DIR ="./papers/"
#VD_DIR ="./src/vectordb/"
QDRANT_PORT=6333


In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv()

True

In [4]:
from huggingface_hub import login, snapshot_download
login(token=os.environ["HUGGING_FACE_HUB_TOKEN"])

snapshot_download(repo_id=OPEN_MODEL ,cache_dir = os.environ["HF_DIR"] + "hub/")#, force_download=True   )

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Fetching 25 files:   0%|          | 0/25 [00:00<?, ?it/s]

'/mnt/jaspero_disk/glycanage/huggingface/hub/models--google--gemma-3-27b-it/snapshots/005ad3404e59d6023443cb575daa05336842228a'

In [5]:
MODELS = [
    {"model_name": "gpt-4o-2024-11-20", "model_short": "gpt-4o", "port": "9999", "text_vd": "RAG_TEXT", "mm_vd": "MM_04_GPT_4o", "late_inter": LI_MODEL, "late_inter_short": LI_MODEL_SHORT }, 
    {"model_name": "gpt-4o-mini-2024-07-18", "model_short": "gpt-4o-mini", "port": "9999", "text_vd": "RAG_TEXT", "mm_vd": "MM_05_GPT_4o_mini", "late_inter": LI_MODEL, "late_inter_short": LI_MODEL_SHORT}, 
    {"model_name": OPEN_MODEL, "model_short": OPEN_MODEL_SHORT, "port": "8006", "text_vd": "RAG_TEXT", "mm_vd": "MM_07_GEMMA3_27B", "late_inter": LI_MODEL, "late_inter_short": LI_MODEL_SHORT}]

In [6]:
papers = [os.path.join(PAPERS_DIR, f) for f in sorted(os.listdir(PAPERS_DIR)) if f.lower().endswith('.pdf')]

In [7]:
#add doi manually!!! or empty list same length as papers
doi_papers = [
"https://doi.org/10.1038/s41590-024-01916-8",
"https://doi.org/10.1186/s12967-018-1695-0",
"https://doi.org/10.1097/hjh.0000000000002963",
"https://doi.org/10.1186/s12967-018-1616-2",
"https://doi.org/10.3390%2Fbiom13020375",
"https://doi.org/10.1016/j.bbagen.2017.06.020",
"https://doi.org/10.1172%2Fjci.insight.89703",
"https://doi.org/10.1016%2Fj.isci.2022.103897",
"https://doi.org/10.1002%2Fart.39273",
"https://doi.org/10.1016/j.bbadis.2018.03.018",
"https://doi.org/10.1097/MIB.0000000000000372",
"https://doi.org/10.1053%2Fj.gastro.2018.01.002",
"https://doi.org/10.1186/s13075-017-1389-7",
"https://doi.org/10.1021/pr400589m",
"https://doi.org/10.1161/CIRCRESAHA.117.312174",
"https://doi.org/10.2337/dc22-0833",
"https://doi.org/10.1097%2FMD.0000000000003379",
"https://doi.org/10.1158/1078-0432.CCR-15-1867",
"https://doi.org/10.1093/gerona/glt190",
"https://doi.org/10.1111/imr.13407",
"https://doi.org/10.1053/j.gastro.2018.05.030",
"https://doi.org/10.1016/j.csbj.2024.03.008",
"https://doi.org/10.1016/j.cellimm.2018.07.009",
"https://doi.org/10.1016/j.biotechadv.2023.108169",
"https://doi.org/10.4049/jimmunol.2400447"
]
#doi_papers = ["" for _ in papers]



In [8]:
processed_multi, processed_text = pdf_loader (papers, doi_papers, [paper.split("/")[-1] for paper in papers],  os.environ["VD_DIR"], emb_tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (516 > 512). Running this sequence through the model will result in indexing errors


For paper_01.pdf there were 93 texts, 10 tables and 32 images processed (needs VLM descriptions), equals 135 documents.
For paper_02.pdf there were 58 texts, 12 tables and 22 images processed (needs VLM descriptions), equals 92 documents.
For paper_03.pdf there were 40 texts, 6 tables and 4 images processed (needs VLM descriptions), equals 50 documents.
For paper_04.pdf there were 51 texts, 11 tables and 11 images processed (needs VLM descriptions), equals 73 documents.
For paper_05.pdf there were 61 texts, 11 tables and 28 images processed (needs VLM descriptions), equals 100 documents.
For paper_06.pdf there were 52 texts, 22 tables and 46 images processed (needs VLM descriptions), equals 120 documents.
For paper_07.pdf there were 31 texts, 0 tables and 19 images processed (needs VLM descriptions), equals 50 documents.
For paper_08.pdf there were 51 texts, 5 tables and 28 images processed (needs VLM descriptions), equals 84 documents.
For paper_09.pdf there were 45 texts, 12 tables a

In [8]:
import pickle

with open('prompts_used.pkl', 'rb') as f:
    prompt = pickle.load(f)

In [10]:
# #placeholder- modify as needed
# prompt = 'You are an expert in glycan biology and you will be querried. Here is the query: {query}\nTask:\nAnswer the clearly and concisely. You will be given Context information, which can be empty.\n Tone: scientific and concise. It should include critical numeric data, significant results, and relevant keywords if relevant.\nConstraints:\nAvoid generic answerd.\nHere is the Context information: \n'

In [11]:
import asyncio 

dict_outputs = await process_models (processed_multi, prompt, MODELS)
dict_outputs["text_only"]= processed_text

print(dict_outputs.keys())

["The input is a figure.  \n\nA circular gauge chart illustrates a variable's performance level, measured in percentages and visually represented as a segment of a circular arc. The color-coded segment highlights progress, with red representing a critical or low-performance range and yellow indicating a moderately cautious zone. Additional data or key metric identifiers are not depicted or legible in this image's current state.", "This figure highlights associations between IgG subclass signatures and disease progression over time. \n\n(a) Odds ratio analysis identifies IgG1 and IgG2 reactivity against specific antigens (e.g., H3N4F1, H5N4F1) linked to disease risk, with significance levels (p-value) represented by circle size and odds ratio by color intensity.  \n(b) Kaplan-Meier curves reveal that higher abundances of IgG1 (H5N4F1, p=0.000097) and IgG2 (H3N4F1, p=0.00012) are correlated with increased complication probability over time.  \n(c-d) Heatmaps of Spearman's correlations fo

['', '(a) The dot plot illustrates significant associations between specific IgG subclasses (e.g., IgG1 H3N4F1 and IgG1 H5N4F1) and disease complications across different time frames post-diagnosis. Notably, IgG1 H3N4F1 and IgG1 H5N4F1 exhibited p-values of 0.00012 and 0.000097, respectively, highlighting the importance of these biomarkers in predicting complications. The odds ratios for associations indicate variability in IgG subtype prevalence, with some demonstrating strong correlations with patient outcomes.\n\n(b) Kaplan-Meier survival curves show distinct complication probabilities linked to high vs. low abundances of IgG1 H3N4F1 and IgG1 H5N4F1 over time, reaffirming statistical significance.\n\n(c, d) Spearman’s correlation matrices across various time intervals (Dx, -1 to -2 years, -2 to -4 years, and -6 years) reveal fluctuating correlations among immune markers, with varying strengths indicated from 1.0 (perfect correlation) to -1.0 (perfect inverse).\n\n(e) A network diagr

In [12]:
import pickle

with open('dict_out.pkl', 'wb') as file:
    pickle.dump(dict_outputs, file)

In [9]:
with open('dict_out.pkl', 'rb') as file:
    dict_outputs=pickle.load( file)

In [10]:
#colpali and similar model/processor retriever
# colpali modules
#poetry add git+https://github.com/illuin-tech/colpali.git
#from colpali_engine.models import ColIdefics3, ColIdefics3Processor
#from colpali_engine.models import ColQwen2_5Omni, ColQwen2_5OmniProcessor
#from colpali_engine.models import ColQwen2_5, ColQwen2_5_Processor

#poetry add git+https://github.com/AhmedMasryKU/colflor.git
from colpali_engine.models import ColPali, ColPaliProcessor
#from colpali_engine.models import ColFlor, ColFlorProcessor

from transformers import AutoProcessor, AutoModel
from transformers.utils.import_utils import is_flash_attn_2_available

def get_ret_model_processor (MODEL_RET, device):
    
    if MODEL_RET == "vidore/colpali-v1.3-merged":
        processor = ColPaliProcessor.from_pretrained(MODEL_RET)

        model = ColPali.from_pretrained(
            MODEL_RET,
            torch_dtype=torch.bfloat16,
            device_map=device,
            attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
        ).eval()
    elif MODEL_RET == "ahmed-masry/ColFlor":
        model = ColFlor.from_pretrained(
            MODEL_RET,
            device_map=device,
            attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
        ).eval()

        processor = ColFlorProcessor.from_pretrained(MODEL_RET)
    
    elif MODEL_RET == "vidore/colSmol-500M":
        model = ColIdefics3.from_pretrained(
            MODEL_RET,
            device_map=device,
            attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
        ).eval()

        processor = ColIdefics3Processor.from_pretrained(MODEL_RET)

    elif MODEL_RET == "ibm-granite/granite-vision-3.3-2b-embedding":
        model = AutoModel.from_pretrained(
            MODEL_RET,
            device_map=device,
            attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
        ).eval()

        processor = AutoProcessor.from_pretrained(MODEL_RET)
        
    elif MODEL_RET == "vidore/colqwen2.5-v0.2":
        model = ColQwen2_5.from_pretrained(
            MODEL_RET,
            torch_dtype=torch.bfloat16,
            device_map=device,  # or "mps" if on Apple Silicon
            attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
            ).eval()
        processor = ColQwen2_5_Processor.from_pretrained(MODEL_RET)
    else:
        print(f"Select correct MODEL_RET, current {MODEL_RET} not correct")
        return -1
    return model, processor

In [11]:
from pdf2image import convert_from_path
from torch.utils.data import DataLoader
from tqdm import tqdm
import uuid
from qdrant_client.http import models

def convert_pdf_to_images(pdf_dir):
    """
    Converts all PDFs in a directory to images.

    Args:
        pdf_dir (str): Path to the directory containing PDFs.

    Returns:
        dict: A dictionary where keys are file names (without extension), 
              and values are lists of images (one list per PDF).
    """
    pdf_list = [pdf for pdf in sorted(os.listdir(pdf_dir)) if pdf.endswith(".pdf")]
    all_images = {}

    for pdf_file in pdf_list:
        pdf_path = os.path.join(pdf_dir, pdf_file)
        pdf_images = convert_from_path(pdf_path)
        all_images[pdf_file] = pdf_images  # Use file name as key
    
    return all_images

def create_document_embeddings(pdf_dir, model, processor, batch_size=2):
    """
    Converts all PDFs in a directory into embeddings with metadata.

    Args:
        pdf_dir (str): Directory containing PDF files.
        model: Pre-trained model for generating embeddings.
        processor: Preprocessor for the model (e.g., to process images).
        batch_size (int): Batch size for inference.

    Returns:
        list: A list of dictionaries, where each dictionary contains:
            - "embedding": The embedding tensor.
            - "doc_id": The document ID (int).
            - "page_id": The page index within the document.
            - "file_name": The name of the source PDF file.
    """
    all_images = convert_pdf_to_images(pdf_dir)
    all_embeddings_with_metadata = []

    for doc_id, (file_name, pdf_images) in enumerate(all_images.items()):
        dataloader = DataLoader(
            dataset=pdf_images,
            batch_size=batch_size,
            shuffle=False,
            collate_fn=lambda x: processor.process_images(x),
        )

        page_counter = 1
        for batch in tqdm(dataloader, desc=f"Processing {file_name}"):
            with torch.no_grad():
                batch = {k: v.to(model.device) for k, v in batch.items()}
                batch_embeddings = model(**batch)
                batch_embeddings = list(torch.unbind(batch_embeddings.to("cpu")))

                for embedding in batch_embeddings:
                    all_embeddings_with_metadata.append(
                        models.PointStruct(
                            id=str(uuid.uuid4()),
                            vector= embedding,
                            payload = {
                              "metadata": {
                                "doc_id": doc_id,
                                "page_id": page_counter,
                                "file_name": file_name, 
                                "img_link":f"{os.environ["VD_DIR"]}pg_images/{file_name.split(".")[0]}_{page_counter:03d}.png",
                                  # Correctly use the file name
                              }    
                            }

                    )
                )

                page_counter += 1

    return all_embeddings_with_metadata


In [12]:
from qdrant_client.http import models

#qdrant should be running in docker !
qdrant_client = QdrantClient(url=f"http://localhost:{QDRANT_PORT}", api_key=os.environ["QDRANT_API_KEY"])

id_text_rag = 0
for idx, model in enumerate(MODELS):
    if idx == 0 and id_text_rag == 0:
        collection_name = model["text_vd"]
        if not qdrant_client.collection_exists(collection_name=collection_name):
            qdrant_client.create_collection(
                collection_name=collection_name,
                on_disk_payload=True,
                vectors_config=models.VectorParams(size=EMB_DIM, on_disk=True, distance=models.Distance.COSINE),
            )
        QdrantVectorStore.from_documents(
            documents=dict_outputs['text_only'],
            url=f"http://localhost:{QDRANT_PORT}",
            api_key=os.environ["QDRANT_API_KEY"],
            collection_name=collection_name,
            embedding=embeddings,
            )
        id_text_rag = 1        

    if not qdrant_client.collection_exists(collection_name=model["mm_vd"]):
        qdrant_client.create_collection(
                collection_name=model["mm_vd"],
                on_disk_payload=True,
                vectors_config= models.VectorParams(
                    size=EMB_DIM,
                    on_disk=True, 
                    distance=models.Distance.COSINE
                    ),
        )
    QdrantVectorStore.from_documents(
    documents=dict_outputs[model["model_short"]],
    url=f"http://localhost:{QDRANT_PORT}",
    api_key=os.environ["QDRANT_API_KEY"],
    collection_name=model["mm_vd"],
    embedding=embeddings,
    )

    if not qdrant_client.collection_exists(collection_name=model["late_inter_short"]):
        qdrant_client.create_collection(
            collection_name=model["late_inter_short"],
            on_disk_payload=True,
            vectors_config=models.VectorParams(
                size=VECTOR_SIZE,
                distance=models.Distance.COSINE,
                on_disk=True,
                multivector_config=models.MultiVectorConfig(
                    comparator=models.MultiVectorComparator.MAX_SIM
                ),
            ),
        )
    model_li, processor_li = get_ret_model_processor (model["late_inter"], "cuda")
    points = create_document_embeddings(PAPERS_DIR, model_li, processor_li, batch_size=4)
    batch_size = 10
    batch_list = [points[n:n+batch_size] for n in range(0, len(points), batch_size)]

    for batch_el in batch_list:
        qdrant_client.upsert(collection_name=model["late_inter_short"], points=batch_el)
    #all_images = convert_pdf_to_images(PAPERS_DIR)


/mnt/jaspero_disk/glycanage/tmp/ipykernel_3182142/519717971.py:4: UserWarning: Api key is used with an insecure connection.
  qdrant_client = QdrantClient(url=f"http://localhost:{QDRANT_PORT}", api_key=os.environ["QDRANT_API_KEY"])
/mnt/jaspero_disk/glycanage/conda_envs/mm_rag_gpu_3.12/lib/python3.12/site-packages/langchain_qdrant/qdrant.py:808: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(**client_options)
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing paper_25.pdf: 100%|██████████| 4/4 [00:02<00:00,  1.63it/s]
/mnt/jaspero_disk/glycanage/conda_envs/mm_rag_gpu_3.12/lib/python3.12/site-packages/langchain_qdrant/qdrant.py:808: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(**client_options)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing paper_25.pdf: 100%|██████████| 4/4 [00:02<00:00,  1.62it/s]
/mnt/jaspero_disk/glycanage/conda_envs/mm_rag_gpu_3.12/lib/python3.12/site-packages/langchain_qdrant/qdrant.py:808: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(**client_options)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing paper_25.pdf: 100%|██████████| 4/4 [00:02<00:00,  1.63it/s]
